In [1]:
import cv2
import numpy as np

# функция для сегментации руки на изображении
def segment(frame, threshold=25):
    # преобразуем изображение в оттенки серого и размываем его
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (7, 7), 0)

    # считываем фоновое изображение
    bg = cv2.imread("background.jpg")
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2GRAY)
    bg = cv2.GaussianBlur(bg, (7, 7), 0)

    # вычисляем разность между текущим кадром и фоном
    diff = cv2.absdiff(bg, gray)

    # пороговая обработка для получения черно-белого изображения
    _, thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # находим контуры на изображении
    _, contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # если контуры найдены, выбираем наибольший контур (руку)
    if len(contours) > 0:
        hand_contour = max(contours, key=cv2.contourArea)

        # аппроксимируем контур
        epsilon = 0.0005 * cv2.arcLength(hand_contour, True)
        approx = cv2.approxPolyDP(hand_contour, epsilon, True)

        # вычисляем выпуклую оболочку контура
        hull = cv2.convexHull(hand_contour)

        # определяем координаты центра масс контура
        M = cv2.moments(hand_contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])

        # определяем количество пальцев на руке
        fingers = 0
        if len(hull) > 3:
            defects = cv2.convexityDefects(hand_contour, cv2.convexHull(hand_contour, returnPoints=False))
            if defects is not None:
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    start = tuple(hand_contour[s][0])
                    end = tuple(hand_contour[e][0])
                    far = tuple(hand_contour[f][0])
                    a = np.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                    b = np.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                    c = np.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                    angle = np.arccos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))
                    if angle <= np.pi / 2:
                        fingers += 1
            # рисуем на изображении контуры, выпуклую оболочку и центр масс
        cv2.drawContours(frame, [hand_contour], 0, (0, 255, 0), 2)
        cv2.drawContours(frame, [hull], 0, (0, 0, 255), 3)
        cv2.circle(frame, (cX, cY), 7, (255, 255, 255), -1)

        # выводим на экран количество пальцев на руке
        cv2.putText(frame, "Fingers: {}".format(fingers), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    return frame

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)

    # вызываем функцию для сегментации руки на изображении и определения количества пальцев
    frame = segment(frame)

    cv2.imshow("Hand Gesture Recognition", frame)

    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()




error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [6]:
import cv2
import numpy as np
import cv2


def segment(frame, threshold=25):
    # Get the background
    bg = cv2.getBackgroundSubtractorMOG2()
    # Process the frame with background subtraction
    fgmask = bg.apply(frame)
    # Apply thresholding on the background subtracted frame
    _, thresholded = cv2.threshold(fgmask, threshold, 255, cv2.THRESH_BINARY)
    # Find the contours in the thresholded frame
    _, contours, _ = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # If no contours are detected, return None
    if len(contours) == 0:
        return None
    else:
        # Find the largest contour
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        # Return the hand segment
        return (thresholded, hand_segment_max_cont)


# Start capturing the video
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # Flip the frame horizontally (mirror effect)
    frame = cv2.flip(frame, 1)
    # Clone the frame
    frame_copy = frame.copy()
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Segment the hand region
    hand = segment(gray)
    # Check if we have a valid hand segment
    if hand is not None:
        # Unpack the thresholded hand segment and the contour
        thresholded, hand_segment = hand
        # Draw contours around hand segment
        cv2.drawContours(frame_copy, [hand_segment + (0, 0)], -1, (0, 255, 255), 2)
        # Count fingers
        finger_count = count_fingers(thresholded, hand_segment, frame_copy)
        # Display finger count
        cv2.putText(frame_copy, str(finger_count), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        # Display thresholded hand segment in separate window
        cv2.imshow("Thresholded Hand Segment", thresholded)
    # Display the hand countour and finger count in separate window
    cv2.imshow("Finger Count",frame_copy)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and destroy all windows
cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2.bgsegm'